In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.optimize import curve_fit


In [2]:
BASE_DIR = '/Users/yanyu/Desktop/课题组文章/金晓强老师/AS审稿意见/20251102代码'
summary_path = os.path.join(BASE_DIR, 'sensitive_analysis', 'sensitive_analysis_summary.csv')
summary_df = pd.read_csv(summary_path)
max_steps = 60000
summary_df.head()


,Lx,Ly,afr,threshold,fifty_step,full_step,step_minus20,step_plus20,step_minus_2pct_full,step_plus_2pct_full
0,40,40,0.10,0.65,817,2358,797,837,770,864
1,40,40,0.10,0.80,1509,3223,1489,1529,1445,1573
2,40,40,0.10,0.95,3618,5360,3598,3638,3511,3725
3,40,40,0.25,0.65,327,944,307,347,309,345
4,40,40,0.25,0.80,604,1289,584,624,579,629


In [3]:
def power_law(x, a, b):
    return a * np.power(np.abs(x), b)


def exp_func(x, a, b):
    return a * np.exp(b * x)



In [4]:
def format_afr(value: float) -> str:
    txt = f"{value:.2f}"
    txt = txt.rstrip('0').rstrip('.')
    return txt


def build_filename(Lx: int, Ly: int, afr: float, step: int) -> str:
    afr_str = format_afr(afr)
    return f"Lx{Lx}_Ly{Ly}_afr{afr_str}_flag1_timestep{step}.csv"


def load_interface_csv(Lx: int, Ly: int, afr: float, step: int):
    filename = build_filename(Lx, Ly, afr, step)
    path = os.path.join(BASE_DIR, 'sensitive_analysis', filename)
    if not os.path.exists(path):
        return None
    data = pd.read_csv(path)
    expected_cols = {'x', 'y', 'angle', 'curvature'}
    if not expected_cols.issubset(data.columns):
        raise ValueError(f"文件 {filename} 缺少必要列 {expected_cols}")
    return data.sort_values('angle').reset_index(drop=True)


def collect_pair_data(first_df: pd.DataFrame, second_df: pd.DataFrame):
    if first_df is None or second_df is None:
        return None
    if len(first_df) != len(second_df):
        return None
    if not np.allclose(first_df['angle'].values, second_df['angle'].values):
        return None

    dx = second_df['x'].values - first_df['x'].values
    dy = second_df['y'].values - first_df['y'].values
    distances = np.sqrt(dx * dx + dy * dy)
    return {
        'angles': first_df['angle'].values,
        'curvatures1': first_df['curvature'].values,
        'curvatures2': second_df['curvature'].values,
        'distances': distances,
        'x1': first_df['x'].values,
        'y1': first_df['y'].values,
        'x2': second_df['x'].values,
        'y2': second_df['y'].values
    }


In [5]:
def analyze_pair(Lx, Ly, afr, threshold, step1, step2, pair_data, output_dir, pair_label):
    angles = pair_data['angles']
    curvatures1 = pair_data['curvatures1']
    curvatures2 = pair_data['curvatures2']
    distances = pair_data['distances']
    x1, y1 = pair_data['x1'], pair_data['y1']
    x2, y2 = pair_data['x2'], pair_data['y2']

    # 相关系数基于前一时间步的曲率
    pearson_r, pearson_p = stats.pearsonr(curvatures1, distances)
    spearman_r, spearman_p = stats.spearmanr(curvatures1, distances)

    # 多项式拟合
    poly1_coefs = np.polyfit(curvatures1, distances, 1)
    poly1_func = np.poly1d(poly1_coefs)

    poly2_coefs = np.polyfit(curvatures1, distances, 2)
    poly2_func = np.poly1d(poly2_coefs)

    poly3_coefs = np.polyfit(curvatures1, distances, 3)
    poly3_func = np.poly1d(poly3_coefs)

    # 幂律拟合
    power_fit_valid = False
    try:
        valid_idx = curvatures1 > 0
        if np.sum(valid_idx) > 2:
            a_power, b_power = curve_fit(power_law, curvatures1[valid_idx], distances[valid_idx], maxfev=10000)[0]
            power_fit_valid = True
        else:
            a_power = b_power = None
    except Exception:
        a_power = b_power = None

    # 指数拟合
    exp_fit_valid = False
    try:
        a_exp, b_exp = curve_fit(exp_func, curvatures1, distances, maxfev=10000)[0]
        exp_fit_valid = True
    except Exception:
        a_exp = b_exp = None

    # 绘图：左边界面，右侧拟合
    fig, (ax_left, ax_right) = plt.subplots(1, 2, figsize=(14, 5))

    sc1 = ax_left.scatter(x1, y1, c=curvatures1, cmap='jet', s=20, label=f'Step {step1}')
    ax_left.plot(x1, y1, 'b-', alpha=0.4)
    sc2 = ax_left.scatter(x2, y2, c=curvatures2, cmap='jet', s=20, marker='s', label=f'Step {step2}')
    ax_left.plot(x2, y2, 'r-', alpha=0.4)
    ax_left.set_xlim(Lx / 2 - 1, Lx)
    ax_left.set_ylim(Ly / 2 - 1, Ly)
    ax_left.set_aspect('equal')
    ax_left.grid(True)
    ax_left.plot(Lx / 2, Ly / 2, 'ko', markersize=4)
    ax_left.axhline(y=Ly / 2, color='gray', linestyle='--', alpha=0.3)
    ax_left.axvline(x=Lx / 2, color='gray', linestyle='--', alpha=0.3)
    cbar = fig.colorbar(sc1, ax=ax_left)
    cbar.set_label('Curvature')
    ax_left.legend(loc='lower left')
    ax_left.set_title(f'Interface ({step1} → {step2})')

    ax_right.scatter(curvatures1, distances, alpha=0.5, s=12, label='Data points')
    x_range = np.linspace(curvatures1.min(), curvatures1.max(), 200)
    ax_right.plot(x_range, poly1_func(x_range), 'r-', linewidth=2, label='Linear')
    ax_right.plot(x_range, poly2_func(x_range), 'g-', linewidth=2, label='Quadratic')
    ax_right.plot(x_range, poly3_func(x_range), 'b-', linewidth=2, label='Cubic')

    if power_fit_valid:
        ax_right.plot(x_range, power_law(x_range, a_power, b_power), 'm-', linewidth=2, label='Power')
    if exp_fit_valid:
        ax_right.plot(x_range, exp_func(x_range, a_exp, b_exp), 'c-', linewidth=2, label='Exp')

    text_info = (f"Pearson r: {pearson_r:.4f} (p={pearson_p:.4f})\n"
                 f"Spearman r: {spearman_r:.4f} (p={spearman_p:.4f})")
    ax_right.text(0.05, 0.95, text_info, transform=ax_right.transAxes, fontsize=11,
                  verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
    ax_right.grid(True)
    ax_right.set_xlabel('Curvature')
    ax_right.set_ylabel('Distance')
    ax_right.legend(loc='lower right')
    ax_right.set_title('Curvature vs. Distance')

    plt.tight_layout()

    afr_str = format_afr(afr)
    output_name = f"Lx{Lx}_Ly{Ly}_afr{afr_str}_flag1_timestep{step1}_{step2}.png"
    output_path = os.path.join(output_dir, output_name)
    plt.savefig(output_path, dpi=300)
    plt.close(fig)

    return {
        'Lx': Lx,
        'Ly': Ly,
        'afr': afr,
        'threshold': threshold,
        'pair_type': pair_label,
        'step_start': step1,
        'step_end': step2,
        'points': len(curvatures1),
        'pearson_r': pearson_r,
        'pearson_p': pearson_p,
        'spearman_r': spearman_r,
        'spearman_p': spearman_p
    }



In [6]:
output_dir = os.path.join(BASE_DIR, 'sensitive_analysis')
results = []

for _, row in summary_df.iterrows():
    Lx = int(row['Lx'])
    Ly = int(row['Ly'])
    afr = float(row['afr'])
    threshold = float(row['threshold'])
    fifty_step = row['fifty_step']

    if pd.isna(fifty_step):
        print(f"跳过 Lx={Lx}, afr={afr}, threshold={threshold}：缺少 50% 步长")
        continue

    fifty_step = int(fifty_step)
    
    # 从 summary CSV 中读取实际的步长值
    step_minus20 = row.get('step_minus20', np.nan)
    step_plus20 = row.get('step_plus20', np.nan)
    step_minus_2pct = row.get('step_minus_2pct_full', np.nan)
    step_plus_2pct = row.get('step_plus_2pct_full', np.nan)
    
    step_info = [
        ('minus20', step_minus20, fifty_step),
        ('plus20', fifty_step, step_plus20),
        ('minus_2pct_full', step_minus_2pct, fifty_step),
        ('plus_2pct_full', fifty_step, step_plus_2pct)
    ]

    data_cache = {}

    for pair_label, start_step, end_step in step_info:
        if pd.isna(start_step) or pd.isna(end_step):
            continue
        start_step = int(start_step)
        end_step = int(end_step)

        if start_step < 1 or end_step < 1 or end_step > max_steps:
            continue

        if start_step not in data_cache:
            data_cache[start_step] = load_interface_csv(Lx, Ly, afr, start_step)
        if end_step not in data_cache:
            data_cache[end_step] = load_interface_csv(Lx, Ly, afr, end_step)

        data_start = data_cache[start_step]
        data_end = data_cache[end_step]

        pair_data = collect_pair_data(data_start, data_end)
        if pair_data is None:
            print(f"警告: Lx={Lx}, afr={afr}, threshold={threshold} 缺少 {start_step}->{end_step} 数据")
            continue

        result_pair = analyze_pair(Lx, Ly, afr, threshold, start_step, end_step, pair_data, output_dir, pair_label)
        results.append(result_pair)
        print(f"Lx={Lx}, afr={afr}, threshold={threshold}, type={pair_label}, steps {start_step}->{end_step}")
        print(f"  数据点: {result_pair['points']}")
        print(f"  Pearson correlation: r = {result_pair['pearson_r']:.6f}, p = {result_pair['pearson_p']:.6f}")
        print(f"  Spearman correlation: r = {result_pair['spearman_r']:.6f}, p = {result_pair['spearman_p']:.6f}\n")

results_df = pd.DataFrame(results)
results_df

Lx=40, afr=0.1, threshold=0.65, type=minus20, steps 797->817
  数据点: 91
  Pearson correlation: r = 0.954162, p = 0.000000
  Spearman correlation: r = 0.979419, p = 0.000000

Lx=40, afr=0.1, threshold=0.65, type=plus20, steps 817->837
  数据点: 91
  Pearson correlation: r = 0.967447, p = 0.000000
  Spearman correlation: r = 0.982513, p = 0.000000

警告: Lx=40, afr=0.1, threshold=0.65 缺少 770->817 数据
警告: Lx=40, afr=0.1, threshold=0.65 缺少 817->864 数据
Lx=40, afr=0.1, threshold=0.8, type=minus20, steps 1489->1509
  数据点: 91
  Pearson correlation: r = 0.898512, p = 0.000000
  Spearman correlation: r = 0.953639, p = 0.000000

Lx=40, afr=0.1, threshold=0.8, type=plus20, steps 1509->1529
  数据点: 91
  Pearson correlation: r = 0.951413, p = 0.000000
  Spearman correlation: r = 0.970351, p = 0.000000

警告: Lx=40, afr=0.1, threshold=0.8 缺少 1445->1509 数据
警告: Lx=40, afr=0.1, threshold=0.8 缺少 1509->1573 数据
Lx=40, afr=0.1, threshold=0.95, type=minus20, steps 3598->3618
  数据点: 91
  Pearson correlation: r = 0.9062

,Lx,Ly,afr,threshold,pair_type,step_start,step_end,points,pearson_r,pearson_p,spearman_r,spearman_p
0,40,40,0.10,0.65,minus20,797,817,91,0.954162,2.080123e-48,0.979419,1.203501e-63
1,40,40,0.10,0.65,plus20,817,837,91,0.967447,6.741734e-55,0.982513,9.139035e-67
2,40,40,0.10,0.80,minus20,1489,1509,91,0.898512,1.400040e-33,0.953639,3.409593e-48
3,40,40,0.10,0.80,plus20,1509,1529,91,0.951413,2.616422e-47,0.970351,1.122664e-56
4,40,40,0.10,0.95,minus20,3598,3618,91,0.906233,4.918131e-35,0.947783,5.971974e-46
5,40,40,0.10,0.95,plus20,3618,3638,91,0.916928,2.847203e-37,0.939638,3.161203e-43
6,40,40,0.25,0.65,minus20,307,327,91,0.980115,2.638316e-64,0.996097,1.266495e-95
7,40,40,0.25,0.65,plus20,327,347,91,0.982767,4.788976e-67,0.997642,2.370607e-105
8,40,40,0.25,0.80,minus20,584,604,91,0.980790,5.762384e-65,0.994130,9.376096e-88
9,40,40,0.25,0.80,plus20,604,624,91,0.975134,4.953020e-60,0.992004,8.450091e-82


In [7]:
pairs_csv_path = os.path.join(output_dir, 'sensitive_analysis_pair_correlations.csv')
results_df.to_csv(pairs_csv_path, index=False)
pairs_csv_path


'/Users/yanyu/Desktop/课题组文章/金晓强老师/AS审稿意见/20251102代码/sensitive_analysis/sensitive_analysis_pair_correlations.csv'